In [33]:
import pandas as pd
import numpy as np

In [34]:
data = pd.read_csv('salary-train.csv')

In [35]:
data['LocationNormalized'].fillna('nan',  inplace = True)
data['ContractTime'].fillna('nan',  inplace = True)
data

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,nan,20355
...,...,...,...,...
59995,"As a result of continued growth, First Class S...",Whitley Bay,contract,26400
59996,PHP / MVC Web Developer MacclesfieldCirca ***...,Macclesfield,permanent,26000
59997,"Staff Nurse, Nursing Home, Baldock White Recru...",Baldock,nan,24500
59998,This is one of the best agency side opportunit...,The City,permanent,65000


In [36]:
#Чтение данных
description = data.iloc[:,0].to_numpy()
location = data.iloc[:, 1].to_numpy()
contracttime = data.iloc[:,2].to_numpy()
salary = data.iloc[:,3].to_numpy()

In [37]:
import re

In [38]:
# Заменить лишние символы в описаниях пробелами и преобразовать к нижнему регистру
for i in range(len(description)):
    description[i] = re.sub('[^a-zA-Z0-9]', ' ', description[i].lower())
    location[i] = location[i].lower()

In [39]:
#Формирование TF-IDF матрицы
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df = 5)
X = vectorizer.fit_transform(description)

In [40]:
# One hot encoding
from sklearn.feature_extraction import DictVectorizer
enc = DictVectorizer()
X_train_categ = enc.fit_transform(data[['LocationNormalized', 'ContractTime']].to_dict('records'))
#X_test_categ = enc.transform(data[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [41]:
from scipy.sparse import hstack
matr = hstack([X, X_train_categ])

In [42]:
from sklearn.linear_model import Ridge
clf = Ridge(alpha=1.0)
clf.fit(matr, salary)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [43]:
mini_data = pd.read_csv('salary-test-mini.csv')

In [44]:
description_test = mini_data.iloc[:,0].to_numpy()
location_test = mini_data.iloc[:, 1].to_numpy()
contracttime_test = mini_data.iloc[:,2].to_numpy()
salary_test = mini_data.iloc[:,3].to_numpy()

In [45]:
for i in range(len(description_test)):
    description_test[i] = re.sub('[^a-zA-Z0-9]', ' ', description_test[i].lower())
    location_test[i] = location_test[i].lower()

In [46]:
len(description_test)

2

In [48]:
X_test = vectorizer.transform(description_test)
X_test_categ = enc.transform(mini_data[['LocationNormalized', 'ContractTime']].to_dict('records'))

In [49]:
X_test_categ.shape

(2, 1766)

In [50]:
X_test.shape

(2, 22861)

In [51]:
matr_test = hstack([X_test, X_test_categ])

In [52]:
clf.predict(matr_test)

array([56567.4617097 , 37189.01395523])